<a href="https://colab.research.google.com/github/Samyadel123/sign-language-project-/blob/main/sign_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

upload you kaggle api

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

downloding the dataset

In [ ]:
!kaggle datasets download risangbaskoro/wlasl-processed

In [ ]:
!unzip /content/wlasl-processed.zip

In [ ]:
def loud_json(file_name:str):
  import json
  with open(file_name,"r") as f:
    data = json.load(f)
  return data

code to see the vido

In [ ]:
import cv2
import time
# Import the Colab-specific display function and utility for clearing output
from google.colab.patches import cv2_imshow
from IPython.display import clear_output

# Set the path to your video file
video_path = "/content/videos/00335.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
else:
    # Get the original FPS for a smoother display (optional but recommended)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps > 0:
        delay = 1 / fps
    else:
        delay = 0.033 # Default to ~30 FPS if not found

    # Loop through the video frames
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            # If the frame was not read successfully, break the loop
            break

        # 1. Display the frame using the Colab-specific function
        cv2_imshow(frame)

        # 2. Clear the previous output cell to simulate video
        # The 'wait=True' ensures a smooth transition
        #clear_output(wait=True)

        # Optional: Add a small sleep to regulate frame rate (improves playback quality)
        time.sleep(delay)

    # Release the VideoCapture object
    cap.release()
    print("Video playback finished.")

# Note: cv2.destroyAllWindows() is NOT needed in Colab.

In [ ]:
!git clone https://github.com/AssemblyAI-Examples/mediapipe-python.git
!pip install mediapipe
!pip install PyQt5
!pip install ipython==7.32.0

In [ ]:
import mediapipe as mp
def mediapip_dedection(fram,model):
  fram = cv2.cvtColor(fram,cv2.COLOR_BGR2RGB) # color conversion for mediapipe
  fram.flags.writeable = False
  results = model.process(fram)
  fram.flags.writeable = True
  fram = cv2.cvtColor(fram,cv2.COLOR_RGB2BGR)
  return fram,results

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils



In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    return image

In [ ]:
def draw_stayled_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,44,121),thickness=1,circle_radius=1)
                              )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(121,44,250),thickness=1,circle_radius=1)
                              )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=1,circle_radius=1)
                              )
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                            )

    return image

In [ ]:
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  cap = cv2.VideoCapture("/content/videos/00335.mp4")
  while cap.isOpened():
    ret,frame = cap.read()
    if not ret:
      break
    # make dedection
    image, result = mediapip_dedection(frame,holistic)

    # draw landmarks
    image = draw_stayled_landmarks(image,result)
    cv2_imshow(image)
    #
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  cap.release()
  cv2.destroyAllWindows()

now i know that the result landmark has four things
- pos landmark
- left-hand landmark
- right-hand landmark
- face landmarks

for example pos landmark is a list for each fram in the vedio

and each one has four componats (x,y,z,visabilty)

In [ ]:
result.pose_landmarks.landmark[0]

In [ ]:
result.face_landmarks.landmark[0]

extract medipipe points in one fram using this function

In [ ]:
import numpy as np
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


In [ ]:
import cv2

def extract_frame_sequence(video_path, start_frame, end_frame, output_prefix="frame",output_path='/content'):
    """
    Extracts a sequence of frames from a video using OpenCV.

    Args:
        video_path (str): Path to the video file.
        start_frame (int): The index of the starting frame (inclusive).
        end_frame (int): The index of the ending frame (inclusive).
        output_prefix (str): Prefix for the saved frame filenames.
    """
    # 1. Open the video
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    if end_frame == -1:
      end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1

    # Check if the end frame is before the start frame
    if end_frame < start_frame:
        print("Error: End frame cannot be less than the start frame.")
        cap.release()
        return

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}")

    # Check if frame numbers are within bounds
    if end_frame >= total_frames:
        print(f"Warning: End frame {end_frame} is beyond the total frames. Setting end_frame to {total_frames - 1}")
        end_frame = total_frames - 1

    # 2. Set the Start Frame
    # cv2.CAP_PROP_POS_FRAMES is the 0-based index of the frame to be decoded/captured next.
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    current_frame_index = start_frame
    frame_count = 0

    print(f"Starting extraction from frame {start_frame} to {end_frame}...")
    with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
      # 3. Loop and Read
      while cap.isOpened() and current_frame_index <= end_frame:
          # Read the next frame
          ret, frame = cap.read()

          if ret:
              # 4. Save/Process the frame
              # Example: Save the frame as a PNG file
              # filename = f"{output_prefix}_{current_frame_index:05d}.png"
              # filename = os.path.join(base_path,filename)
              filename = os.path.join(output_path,f"{output_prefix}_{current_frame_index:05d}.npy")
              # medipipe dedection
              image, results = mediapip_dedection(frame,holistic)
              # extract keypoints
              keypoints = extract_keypoints(results)

              #cv2.imwrite(filename, frame)
              np.save(filename,keypoints)
              # Increment the frame count
              frame_count += 1
              current_frame_index += 1
          else:
              # Break the loop if we can't read the next frame
              print(f"Warning: Could not read frame at index {current_frame_index}. Stopping.")
              break

    # Release the video capture object
    cap.release()
    print(f"Extraction complete. {frame_count} frames saved.")



let's make an experiment

load the config file and get a test exampe to work with

## function to exract imges

In [ ]:
import os

data = loud_json("/content/WLASL_v0.3.json")
one = data[0]
# lable / gloss
gloss = one["gloss"]
# instances
instances = one["instances"]

# get the first vido in the first sample
video_id = instances[0]["video_id"]
frame_start = instances[0]["frame_start"]
frame_end = instances[0]["frame_end"]

# constract vido path
video_path = os.path.join("/content/videos",f"{video_id}.mp4")

output_prefix = gloss



extract the frams i need

In [ ]:
# see a sample frame
import cv2
import matplotlib.pyplot as plt
sample_imge = "/content/book_00049.png"
# display image
img = cv2.imread(sample_imge)
plt.imshow(img)
plt.show()

## setting folder structure

In [ ]:
# now i want to make a dict with the key as the gloss and a list of vido id's for each gloss
import json
from dataclasses import dataclass

@dataclass
class unit:
  video_id:str
  frame_start:int
  frame_end:int

with open("/content/WLASL_v0.3.json","r") as f:
  data = json.load(f)
gloss_vido_dict = {}
for sample in data:
  gloss = sample["gloss"]
  instances = sample["instances"]
  for instance in instances:
    video_id = instance["video_id"]
    frame_start = instance["frame_start"]
    frame_end = instance["frame_end"]
    unit_instance = unit(
        video_id=video_id,
        frame_start=frame_start,
        frame_end=frame_end
    )
    if gloss not in gloss_vido_dict:
      gloss_vido_dict[gloss] = []
    gloss_vido_dict[gloss].append(unit_instance)

In [ ]:
# folder strucre
import os
import shutil

base_exraction_path = "/content/MyData"
# make my data folder
os.makedirs(base_exraction_path,exist_ok=True)

# loop over gloss
for gloss in gloss_vido_dict:
  # loop over vido id
  for unit in gloss_vido_dict[gloss]:
    # make a folder with (/content/mydat/gloss(class name)/vido_id/our exracted frams)
    # make the path to vedio
    video_folder_path = os.path.join(base_exraction_path, gloss, unit.video_id)
    # make the folder
    os.makedirs(video_folder_path,exist_ok=True)
    os.makedirs(video_folder_path, exist_ok=True)

In [ ]:
# now let's do the exraction
for gloss in gloss_vido_dict:
  # loop over vido id
  for unit in gloss_vido_dict[gloss]:
    start_frame = unit.frame_start
    end_frame = unit.frame_end
    video_path = os.path.join("/content/videos",f"{unit.video_id}.mp4")
    output_prefix = os.path.join(base_exraction_path,gloss,unit.video_id)
    output_path = os.path.join(base_exraction_path,gloss,unit.video_id)
    extract_frame_sequence(video_path,start_frame,end_frame,output_prefix,output_path=output_path)

In [ ]:
# save the exracted foldet to mydrive to work on it directly
from google.colab import drive
drive.mount('/content/drive')

!zip -r /content/MyData.zip /content/MyData
!cp /content/MyData.zip /content/drive/MyDrive/

In [ ]:
  # extract land marks form resuls
out=extract_keypoints(results=result)
np.save("/content/test.npy",out)